In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 7.6 MB/s eta 0:00:00


In [1]:
from ultralytics import YOLO


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def detect_objects(model, source, folder, object_type):
    np_model = YOLO(model)
    src = np_model.track(source, persist=True)
    for i, result in enumerate(src):
        result[0].save_crop(folder, file_name=f'{object_type}{i}')
    print('done')

def detect_cars(model, source, folder):
    detect_objects(model, source, folder, 'car')

def detect_plates(model, source, folder):
    detect_objects(model, source, folder, 'plate')

In [4]:
# backbone = YOLO("yolov8n.pt")
detect_cars("yolov8n.pt", '/content/Untitled.jpg', '/content/result')

⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt...
⚠️ Download failure, retrying 2/3 https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt...
⚠️ Download failure, retrying 3/3 https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt...


ConnectionError:   Download failure for https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8n.pt. Retry limit reached.

In [2]:
import os

os.path.abspath("..")

'c:\\dev\\gen projects\\ANPR'

In [3]:
ARTIFACT_PATH = os.path.join(os.path.abspath(".."),"artifacts")
ARTIFACT_PATH

'c:\\dev\\gen projects\\ANPR\\artifacts'

In [8]:
detect_plates(os.path.join(ARTIFACT_PATH,"models","best.torchscript"), os.path.join(ARTIFACT_PATH,"samples","pandemic.jpg"), os.path.join(ARTIFACT_PATH,"results"))

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading c:\dev\gen projects\ANPR\artifacts\models\best.torchscript for TorchScript inference...

image 1/1 c:\dev\gen projects\ANPR\artifacts\samples\pandemic.jpg: 640x640 1 License_Plate, 1367.6ms
Speed: 21.1ms preprocess, 1367.6ms inference, 31.3ms postprocess per image at shape (1, 3, 640, 640)
done


In [25]:
import cv2
import numpy as np
import functools
import matplotlib.pyplot as plt
image = cv2.imread(os.path.join(ARTIFACT_PATH,"results","License_Plate","plate04.jpg"))

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blurred = cv2.GaussianBlur(gray, (1,1), 0)

thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 15)

cv2.imshow("test", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
cv2.__version__

'4.6.0'

In [ ]:
cv2.drawContours(image,)

In [5]:
_, labels = cv2.connectedComponents(thresh)
mask = np.zeros(thresh.shape, dtype="uint8")
cv2.imshow('hh',mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)

In [13]:
total_pixels = image.shape[0] * image.shape[1]
lower = total_pixels // 50
upper = total_pixels // 20
print(total_pixels)

11880


In [14]:
for(i, label) in enumerate(np.unique(labels)):

  if label == 0:
    continue

  labelMask = np.zeros(thresh.shape, dtype="uint8")
  labelMask[labels == label] = 255
  numPixels = cv2.countNonZero(labelMask)


  if numPixels > lower and numPixels < upper :
    mask = cv2.add(mask, labelMask)

cv2.imshow('hh',mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [19]:
cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boundingBoxes = [cv2.boundingRect(c) for c in cnts]


In [21]:
def compare(rect1, rect2):
  if abs(rect1[1] - rect2[1]) > 10:
    return rect1[1] - rect2[1]
  else:
    return rect1[0] - rect2[0]
boundingBoxes = sorted(boundingBoxes, key=functools.cmp_to_key(compare))

In [20]:
boundingBoxes

[]

In [18]:
for bbox in boundingBoxes:
    x, y, w, h = bbox
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('hg',image)
cv2.waitKey(0)
cv2.destroyAllWindows()